## Vector Search with IRIS SQL
This tutorial covers how to use IRIS as a vector database. 

For this tutorial, we will use a dataset of 2.2k online reviews of scotch (
dataset from https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews) . With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the online reviews of scotch whiskeys. In addition, we'll be able to apply filters on columns with structured data. For example, we will be able to search for whiskeys that are priced under $100, and are 'earthy, smooth, and easy to drink'. Let's find our perfect whiskey!

In [1]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

from dotenv import load_dotenv
load_dotenv(override=True)

/opt/homebrew/anaconda3/envs/gs24ws/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [9]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '55665' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [10]:
engine = create_engine(CONNECTION_STRING)

### Load tweet data
Load the json file with tweets into a Pandas DataFrame that can be easily imported into IRIS as a SQL table.

In [3]:
import pandas as pd

# Load JSONL file into DataFrame
file_path = './data/financial/tweets_all.jsonl'
df_tweets = pd.read_json(file_path, lines=True)


In [5]:
df_tweets.head()

,note
0,$BYND - JPMorgan reels in expectations on Beyo...
1,$CCL $RCL - Nomura points to bookings weakness...
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ..."
3,$ESS: BTIG Research cuts to Neutral https://t....
4,$FNKO - Funko slides after Piper Jaffray PT cu...


Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, 'note_vector', will be used to store vectors that are generated by passing the 'note' of a review through an embedding model.

In [11]:

with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE TABLE financial_tweets (
        note VARCHAR(255),
        note_vector VECTOR(DOUBLE, 384)
        )
                """
        result = conn.execute(text(sql))

In [12]:
# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [14]:

# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df_tweets['note'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df_tweets['note_vector'] = embeddings.tolist()


In [15]:
df_tweets.head()

,note,note_vector
0,$BYND - JPMorgan reels in expectations on Beyo...,"[-0.13631078600883484, 0.026333356276154518, -..."
1,$CCL $RCL - Nomura points to bookings weakness...,"[-0.033777981996536255, 0.06702922284603119, -..."
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...","[-0.08540519326925278, 0.04619771987199783, 0...."
3,$ESS: BTIG Research cuts to Neutral https://t....,"[-0.13111060857772827, 0.03535114973783493, 0...."
4,$FNKO - Funko slides after Piper Jaffray PT cu...,"[-0.0776449665427208, 0.055340882390737534, -0..."


In [16]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df_tweets.iterrows():
            sql = text("""
                INSERT INTO financial_tweets 
                (note, note_vector) 
                VALUES (:note, TO_VECTOR(:note_vector))
            """)
            conn.execute(sql, {
                'note': row['note'], 
                'note_vector': str(row['note_vector'])
            })


Let's find some specific tweets

In [17]:
note_search = "covid effect"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector

In [18]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * FROM financial_tweets
            ORDER BY VECTOR_DOT_PRODUCT(note_vector, TO_VECTOR(:search_vector)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()


In [19]:
print(results)

[("New developments added to @FedFRASER's COVID-19 timeline in the latest week: second historic rise in unemployment i… https://t.co/o4yYfNRbhA", '-.085796736180782318116,-.014474455267190933227,.024680249392986297607,.026129452511668205261,.091199100017547607421,.054136261343955993652,-.1042803 ... (8841 characters truncated) ... 7380371,-.0045623378828167915344,-.013932778500020503997,.091904394328594207763,-.086062125861644744873,.053048193454742431641,.088242650032043457031'), ('Central banks must evolve to help governments fight coronavirus https://t.co/mfSJuTKUDm', '.010059537366032600402,-.022754097357392311096,-.017637876793742179871,.0082869371399283409118,.075412169098854064941,.0018180640181526541709,-.03168 ... (8846 characters truncated) ... 9053802,.047726158052682876586,-.031523786485195159912,.041589658707380294799,-.056619014590978622436,.093337520956993103027,-.0020199415739625692367'), ('Luckin Coffee and Yum China hit again by coronavirus anxiety', '.053168803453445

In [20]:
results_df = pd.DataFrame(results, columns=df_tweets.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,note
0,New developments added to @FedFRASER's COVID-19 timeline in the latest week: second historic rise in unemployment i… https://t.co/o4yYfNRbhA
1,Central banks must evolve to help governments fight coronavirus https://t.co/mfSJuTKUDm
2,Luckin Coffee and Yum China hit again by coronavirus anxiety
